In [1]:
import os
import ee
import geemap
import numpy as np

In [2]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [5]:
%load_ext autoreload
%autoreload 2
import sys  
sys.path.insert(0, '../tools/')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
%reload_ext autoreload
# import sand_classification as sc
import quality_mosaic as qm
# import train_val as tv

# Download image by tile

In [16]:
# read a shapefile into a geopandas dataframe
import geopandas as gpd

test_roi = '../data/betwa-test-poly-polygon.shp'
test_kml = '../data/betwa-test-poly.kml'
gdf = gpd.read_file(test_roi)

gdf

,Name,geometry
0,betwa-test-poly,"POLYGON ((79.78787 25.86981, 79.80159 25.87578..."


In [19]:
roi = geemap.vector_to_ee(test_roi)
roi.getInfo()

{'type': 'FeatureCollection',
 'columns': {'Name': 'String', 'system:index': 'String'},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[79.7878656712008, 25.8698057494276],
      [79.7865664306152, 25.8611442683894],
      [79.7939189671744, 25.8609942413027],
      [79.7957159919335, 25.8587917043692],
      [79.8001974852475, 25.8594975302394],
      [79.8045558619593, 25.8622467867835],
      [79.8098113182621, 25.8647280973938],
      [79.8153582342182, 25.8718347987767],
      [79.8187516892406, 25.8803907029506],
      [79.8190886026307, 25.8906248414734],
      [79.8274575552957, 25.8920194681292],
      [79.8367029884974, 25.8830054583251],
      [79.8532044490541, 25.8766566096896],
      [79.8705191819733, 25.8771483229725],
      [79.8804617361309, 25.8817768497765],
      [79.8910458910375, 25.8869824180478],
      [79.9000377450384, 25.8797718428418],
      [79.9069326308557, 25.884540368319],
      [79.8998634663856, 25.894784159

In [37]:
S2_BANDS = ["B1","B2","B3","B4","B5", "B6","B7","B8","B8A", "B9","B11","B12"]

In [41]:
date = '2022-05-07' #from analysis in river_sand_flooding_gst_ganga GEE 
max_search_window_months = 1
median_samples = 5
mosaic_method = 'median'


# mining_centroids = ee.FeatureCollection('projects/gee-sand/assets/bihar_sand_banks_centroids')

s1_s2 = qm.get_s1_s2(roi=roi, date=date, max_search_window_months=max_search_window_months,median_samples=median_samples, mosaic_method=mosaic_method, clip=False, bands=S2_BANDS)
s1_s2.bandNames()
# roi

Search window from 2022-04-07 to 2022-06-07


In [42]:
Map = geemap.Map()
Map.centerObject(roi, 13)
Map.add_basemap('SATELLITE')
# Map.add_basemap('Stamen.Toner')
# Map.addLayer(s1_s2, qm.visParamsVV, 'S1', False)
# Map.addLayer(s1_s2, qm.visParamsMTGSI , 'mTGSI', False)
Map.addLayer(s1_s2, qm.visParamsRGB, 'S2')
Map.addLayer(roi, {'opacity':0.5}, 'roi')

Map

Map(center=[25.888620679819024, 79.83629344744737], controls=(WidgetControl(options=['position', 'transparent_…

## Override the library's fishnet(..) function with a custom one

In [43]:
def fishnet(
    data,
    h_interval=1.0,
    v_interval=1.0,
    rows=None,
    cols=None,
    delta=1.0,
    intersect=True,
    output=None,
    lat_offset=0.,
    lon_offset=0.,
    **kwargs,
):
    """Create a fishnet (i.e., rectangular grid) based on an input vector dataset.

    Args:
        data (str | ee.Geometry | ee.Feature | ee.FeatureCollection): The input vector dataset. It can be a file path, HTTP URL, ee.Geometry, ee.Feature, or ee.FeatureCollection.
        h_interval (float, optional): The horizontal interval in degrees. It will be ignored if rows and cols are specified. Defaults to 1.0.
        v_interval (float, optional): The vertical interval in degrees. It will be ignored if rows and cols are specified. Defaults to 1.0.
        rows (int, optional): The number of rows. Defaults to None.
        cols (int, optional): The number of columns. Defaults to None.
        delta (float, optional): The buffer distance in degrees. Defaults to 1.0.
        intersect (bool, optional): If True, the output will be a feature collection of intersecting polygons. Defaults to True.
        output (str, optional): The output file path. Defaults to None.
        lat_offset (float, optional): The latitude offset in degrees. Defaults to 0.
        lon_offset (float, optional): The longitude offset in degrees. Defaults to 0.

    Returns:
        ee.FeatureCollection: The fishnet as an ee.FeatureCollection.
    """
    if isinstance(data, str):
        data = geemap.vector_to_ee(data, **kwargs)

    if isinstance(data, ee.FeatureCollection) or isinstance(data, ee.Feature):
        data = data.geometry()
    elif isinstance(data, ee.Geometry):
        pass
    else:
        raise ValueError(
            "data must be a string, ee.FeatureCollection, ee.Feature, or ee.Geometry."
        )

    # print("HOTDAMN")
    coords = data.bounds().coordinates().getInfo()

    west = coords[0][0][0] + lon_offset
    east = coords[0][1][0] + lon_offset
    south = coords[0][0][1] + lat_offset
    north = coords[0][2][1] + lat_offset

    if rows is not None and cols is not None:
        v_interval = (north - south) / rows
        h_interval = (east - west) / cols

    # west = west - delta * h_interval
    east = east + delta * h_interval
    # south = south - delta * v_interval
    north = north + delta * v_interval

    grids = geemap.latlon_grid(v_interval, h_interval, west, east, south, north)

    if intersect:
        grids = grids.filterBounds(data)

    if output is not None:
        geemap.ee_export_vector(grids, output)

    else:
        return grids

In [26]:
tile_side_m = 440
tile_stride_m = 220


In [27]:
#1111m = 0.01deg lat, 440m  = 0.01deg * 440/1111
v_grid = 0.01 * tile_side_m/1111
h_grid = v_grid / np.cos(np.deg2rad(lat)) #higher latitudes have larger values for degrees to get the same size in km
v_stride = 0.01 * tile_stride_m/1111
h_stride = v_stride / np.cos(np.deg2rad(lat))

h_grid, v_grid, h_stride, v_stride

(0.0045613873030399855,
 0.0039603960396039604,
 0.0022806936515199928,
 0.0019801980198019802)

In [44]:
grid1 = fishnet(roi, h_interval=h_grid, v_interval=v_grid, delta=0.1, intersect=True)
Map.addLayer(grid1, {'color':'red', 'opacity':0.3}, 'Grid1')

# grid2 = fishnet(roi, h_interval=h_grid, v_interval=v_grid, delta=0.1, intersect=True, lat_offset=0., lon_offset=h_stride)
# Map.addLayer(grid2, {'color':'yellow', 'opacity':0.3}, 'Grid2')

# grid3 = fishnet(roi, h_interval=h_grid, v_interval=v_grid, delta=0.1, intersect=True, lat_offset=v_stride, lon_offset=0.)
# Map.addLayer(grid3, {'color':'blue', 'opacity':0.3}, 'Grid3')

grid4 = fishnet(roi, h_interval=h_grid, v_interval=v_grid, delta=0.1, intersect=True, lat_offset=v_stride, lon_offset=h_stride)
Map.addLayer(grid4, {'color':'white', 'opacity':0.3}, 'Grid4')

In [45]:

# visParams = {"bands": ["B1","B2","B3","B4","B5", "B6","B7","B8","B8A", "B9","B11","B12"]}

baseM = s1_s2.reproject(crs = s1_s2.projection(), scale = 10)#.visualize(**visParams)

baseM.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:4326',
   'crs_transform': [8.983152841195215e-05,
    0,
    0,
    0,
    -8.983152841195215e-05,
    0]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:4326',
   'crs_transform': [8.983152841195215e-05,
    0,
    0,
    0,
    -8.983152841195215e-05,
    0]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:4326',
   'crs_transform': [8.983152841195215e-05,
    0,
    0,
    0,
    -8.983152841195215e-05,
    0]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:4326',
   'crs_transform': [8.983152841195215e-05,
    0,
    0,
    0,
    -8.983152841195215e-05,
    0]},
  {'id': 'B5',
   'da

In [46]:
geemap.download_ee_image_tiles(
    baseM, grid4, '../data/patches/', prefix="s2_", scale=10
)

s2_001.tif: |          | 0.00/225k (raw) [  0.0%] in 00:00 (eta:     ?)

s2_002.tif: |          | 0.00/220k (raw) [  0.0%] in 00:00 (eta:     ?)

s2_003.tif: |          | 0.00/225k (raw) [  0.0%] in 00:00 (eta:     ?)

s2_004.tif: |          | 0.00/225k (raw) [  0.0%] in 00:00 (eta:     ?)

s2_005.tif: |          | 0.00/225k (raw) [  0.0%] in 00:00 (eta:     ?)

...

s2_063.tif: |          | 0.00/220k (raw) [  0.0%] in 00:00 (eta:     ?)

s2_064.tif: |          | 0.00/225k (raw) [  0.0%] in 00:00 (eta:     ?)

s2_065.tif: |          | 0.00/225k (raw) [  0.0%] in 00:00 (eta:     ?)

s2_066.tif: |          | 0.00/225k (raw) [  0.0%] in 00:00 (eta:     ?)

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))